# SFT (Supervised Fine Tuning) for LLM
- 금융언어모델 학습
- instruction tuning
- 추론능력 향상 및 지시에 맞는 답변 포맷팅 목적

## 모델 및 데이터셋 로드

In [1]:
hf_token = 'hf_****'

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1500
# max_seq_length = 1024
# max_seq_length = 2000
dtype = None # None으로 지정할 경우 해당 컴퓨팅 유닛에 알맞은 dtype으로 저장됩니다. Tesla T4와 V100의 경우에는 Float16, Ampere+ 이상의 경우에는 Bfloat16으로 설정됩니다.
load_in_4bit = False # 메모리 사용량을 줄이기 위해서는 4bit 양자화를 사용하실 것을 권장합니다.

# 모델 및 토크나이저 선언
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lsw0570168/krx-q25-7b-base-v7.2-fft",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token, # gated model을 사용할 경우 허깅페이스 토큰을 입력해주시길 바라겠습니다.
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.2: Fast Qwen2 patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA H100 80GB HBM3. Max memory: 79.109 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
# lora 학습시 활성화

# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 32,  # 16, # 0을 넘는 숫자를 선택하세요. 8, 16, 32, 64, 128이 추천됩니다.
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",], # target module도 적절하게 조정할 수 있습니다.
#     lora_alpha = 16,
#     lora_dropout = 0, # 어떤 값이든 사용될 수 있지만, 0으로 최적화되어 있습니다.
#     bias = "none",    # 어떤 값이든 사용될 수 있지만, "none"으로 최적화되어 있습니다.
#     use_gradient_checkpointing = "unsloth", # 매우 긴 context에 대해 True 또는 "unsloth"를 사용하십시오.
#     random_state = 42,
#     use_rslora = False,
#     loftq_config = None
# )

## 데이터셋 전처리

In [4]:
import json
from datasets import load_dataset, concatenate_datasets, Dataset
import random

def load_jsonl(file_path):
    data = []
    err_cnt = 0
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            try:
                data.append(json.loads(line.strip()))
            except Exception as e:
                err_cnt += 1
    if err_cnt > 0:
        print(f"err line count: {err_cnt}")
    return data


with open('./sft_kbench-p.json', 'r') as f:
    dd = json.load(f)

EOS_TOKEN = tokenizer.eos_token
ds = Dataset.from_dict(
    {'formatted_text': [f"{x}{EOS_TOKEN}" for x in dd]}
)#.shuffle(seed=777).select(range(2000))
ds

Dataset({
    features: ['formatted_text'],
    num_rows: 1949
})

In [5]:
print(ds['formatted_text'][0])

다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.
### 질문: 다음 데이터프레임에서 'GDP' 값이 가장 큰 해의 'Year' 값을 구하세요.

### df.head()
|    | Year | GDP      |
|----|------|----------|
|  0 | 2010 | 15000.0  |
|  1 | 2011 | 15500.5  |
|  2 | 2012 | 16000.0  |
|  3 | 2013 | 16250.0  |
|  4 | 2014 | 17000.0  |

### 선택지:
A. df.loc[df['GDP'].idxmax(), 'Year']
B. df[df['GDP'] == df['GDP'].max()]['Year']
C. df['Year'][df['GDP'].idxmax()]
D. df.loc[df['Year'].idxmax(), 'GDP']
E. df[df['GDP'].argmax()]['Year']
F. df[df['GDP'] == df['GDP'].max()].Year.iloc[0]
### 정답: 먼저 차근차근 한 단계씩 생각해봅시다.
#### step1. 문제 이해:
문제는 데이터프레임에서 'GDP' 값이 가장 큰 해의 'Year' 값을 찾는 것입니다. 이는 데이터프레임의 특정 열에서 최대값을 찾고, 그에 해당하는 다른 열의 값을 추출하는 문제로, 판다스 라이브러리를 사용하여 해결할 수 있습니다.
#### step2. 각 보기별 문제 풀이:
각 보기별로 분석해보겠습니다.
A. df.loc[df['GDP'].idxmax(), 'Year'] - 'GDP'의 최대값 인덱스를 찾아 해당 인덱스의 'Year' 값을 반환합니다. 올바른 방법입니다.
B. df[df['GDP'] == df['GDP'].max()]['Year'] - 'GDP'의 최대값과 같은 행을 필터링하여 'Year' 값을 반환합니다. 올바른 방법입니다.
C. df['Year'][df['GDP'].idxmax()] - 'GDP'의 최대값 인덱스를 찾아 'Year' 열에

In [9]:
dd4 = load_jsonl('./qa_all.jsonl')
dd4 = [aaa for aaa in dd4 if len(aaa['question']) > 230]
dd42 = load_jsonl('./error_note.jsonl')
dd4 = dd4 + dd42 + dd42
ds4 = Dataset.from_dict(
    {'formatted_text': [f"{x['question']}\n\n{x['answer']}{EOS_TOKEN}" for x in dd4]}
).shuffle(seed=777)#.select(range(5000))
ds4

Dataset({
    features: ['formatted_text'],
    num_rows: 2200
})

In [14]:
rs = load_dataset("kuotient/orca-math-korean-preference", split='train') \
    .rename_columns({'question': "prompt", "answer": "response"}) \
    .select_columns(['prompt', 'response']) \
    .shuffle(seed=777).select(range(5000))
rs

Dataset({
    features: ['prompt', 'response'],
    num_rows: 5000
})

In [15]:
rs2 = load_dataset("ChuGyouk/argilla-distilabel-math-preference-dpo-korean", split='train') \
    .rename_columns({'instruction_ko': "prompt", "chosen_response_ko": "response", "chosen_response":"response_en"}) \
    .select_columns(['prompt', 'response', 'response_en'])

rr2 = [
    f"다음 한국어 문장을 영어로 번역하시오.\n\n### 한국어:\n{x['response']}\n\n### 영어:\n{x['response_en']}" \
    for x in rs2.to_list()[:int(len(rs2)/2)]
]
rr22 = [
    f"Translate English statement below to Korean.\n\n### English:\n{x['response_en']}\n\n### Korean:\n{x['response']}" \
    for x in rs2.to_list()[int(len(rs2)/2):]
]
tmp = Dataset.from_dict({
    "prompt": ['']*len(rr2+rr22), "response": rr2+rr22
})

rs2 = concatenate_datasets([rs2]) \
    .select_columns(['prompt', 'response']) \
    .shuffle(seed=777)#.select(range(2000))
rs2

Dataset({
    features: ['prompt', 'response'],
    num_rows: 2418
})

In [16]:
rs3 = load_dataset("MarkrAI/KoCommercial-Dataset", split='train') \
    .rename_columns({'instruction':"prompt", "output":"response"}) \
    .select_columns(['prompt', 'response']) \
    .shuffle(seed=777).select(range(5000))
rs3

Dataset({
    features: ['prompt', 'response'],
    num_rows: 5000
})

In [17]:
rs4 = load_dataset("dhruvnathawani/gretelai_synthetic-gsm8k-reflection-405b_validated_with_o1-mini_validation_only", split='train') \
    .rename_columns({'question':"prompt", "answer":"response"}) \
    .select_columns(['prompt', 'response']) \
    .shuffle(seed=777).select(range(5000))
rs4

Dataset({
    features: ['prompt', 'response'],
    num_rows: 5000
})

In [18]:
rs5 = load_dataset("heegyu/CoT-collection-ko", split='train') \
    .select_columns(['source', 'rationale']) \
    .rename_columns({'source':"prompt", "rationale":"response"}) \
    .select_columns(['prompt', 'response']) \
    .shuffle(seed=777).select(range(5000))
rs5

Dataset({
    features: ['prompt', 'response'],
    num_rows: 5000
})

In [19]:
rs6 = load_dataset("daekeun-ml/naver-news-summarization-ko", split='train') \
    .select_columns(['document', 'summary']) \
    .rename_columns({'document':"prompt", "summary":"response"}) \
    .select_columns(['prompt', 'response']) \
    .shuffle(seed=777).select(range(5000))
rr6 = [
    f"다음 뉴스 본문을 요약하시오.\n\n### 뉴스본문:\n{x['prompt']}\n\n### 요약:\n{x['response']}" \
    for x in rs6.to_list()
]
rs6 = Dataset.from_dict({"prompt": ['']*len(rr6), "response": rr6}) \
    .select_columns(['prompt', 'response']) \
    .shuffle(seed=777)#.select(range(2000))
rs6

Dataset({
    features: ['prompt', 'response'],
    num_rows: 5000
})

In [22]:
prompt_format = "{}\n\n{}"
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["prompt"]
    outputs = examples["response"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt_format.format(instruction, output) + EOS_TOKEN # 마지막에 eos token을 추가해줌으로써 모델이 출력을 끝마칠 수 있게 만들어 줍니다.
        # text = 'axf'*10000+prompt_format.format(instruction, output) + EOS_TOKEN # 마지막에 eos token을 추가해줌으로써 모델이 출력을 끝마칠 수 있게 만들어 줍니다.
        texts.append(text)
    return { "formatted_text" : texts, }
pass

ds_comb = concatenate_datasets([rs, rs2, rs3, rs4, rs5, rs6])

_dataset = ds_comb.map(formatting_prompts_func, batched = True,)
_dataset

Map:   0%|          | 0/2250 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'response', 'formatted_text'],
    num_rows: 2250
})

In [23]:
dataset = concatenate_datasets([ds, _dataset]) \
            .select_columns(['formatted_text']) \
            .shuffle(seed=777)

dataset

Dataset({
    features: ['formatted_text'],
    num_rows: 4511
})

In [24]:
print(dataset['formatted_text'][0])

다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.
### 질문: 다음 중 단기매매증권 평가손익에 대한 설명으로 옳은 것은 무엇인가?
### 선택지:
A. 단기매매증권 평가손익은 자본잉여금으로 처리한다.
B. 단기매매증권 평가손익은 기타포괄손익으로 처리한다.
C. 단기매매증권 평가손익은 당기손익으로 처리한다.
D. 단기매매증권 평가손익은 미처분이익잉여금으로 처리한다.
### 정답: 
1. 문제 유형: 정의 기반

2. 문제 풀이 전략:
   - 정의 기반 문제의 경우, 관련 용어의 정의나 원칙을 정확히 이해하고 있어야 합니다. 따라서, 문제에서 언급된 용어(예: 단기매매증권 평가손익)에 대한 회계 기준이나 원칙을 검토하여 올바른 처리를 선택합니다.

3. 세부 풀이 과정:
   - 단기매매증권은 기업이 단기적인 시세 차익을 목적으로 보유하는 금융자산입니다. 이러한 증권은 공정가치로 평가되며, 평가손익은 당기손익으로 인식됩니다.
   - 이는 국제회계기준(IFRS) 및 한국채택국제회계기준(K-IFRS)에서 규정하는 바에 따라, 단기매매증권의 평가손익은 손익계산서에 반영되어 당기손익으로 처리됩니다.
   - 따라서, 보기 중에서 단기매매증권 평가손익을 당기손익으로 처리한다고 설명한 선택지를 찾습니다.

최종적으로, 정답은 C. 단기매매증권 평가손익은 당기손익으로 처리한다.
### 정답: C<|endoftext|>


In [25]:
# from tqdm.notebook import tqdm
# import seaborn as sns

# res3 = []
# for x in tqdm(dataset['formatted_text']):
#     res3.append(len(tokenizer.encode(x)))

# print(len([x for x in res3 if x>max_seq_length]))
# sns.histplot(res3)

In [26]:
# # 너무 긴 데이터는 잘못된 데이터일 가능성 크므로 제외!

# idxs = [i for i,x in enumerate(res3) if x>3000]
# print(len(idxs))

In [27]:
# dataset = dataset.filter(lambda x, i: i not in idxs, with_indices=True)
# dataset

## 모델 학습


In [28]:
import os
import requests
import wandb

# 고정 설정
os.environ['WANDB_PROJECT']="krx"
os.environ['WANDB_API_KEY']="****"
os.environ['WANDB_ENTITY']="****"

# 개별 설정
os.environ['WANDB_TAGS']=""
run_name = ""

In [29]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "formatted_text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 16,
    packing = False, # True로 설정하면 짧은 텍스트 데이터에 대해서는 더 빠른 학습 속도로를 보여줍니다.
    args = TrainingArguments( # TrainingArguments는 자신의 학습 환경과 기호에 따라 적절하게 설정하면 됩니다.
        per_device_train_batch_size = 2,  # A10(22g) - aws
        gradient_accumulation_steps = 8,        
        # per_device_train_batch_size = 48,  # H100(80g)
        # gradient_accumulation_steps = 1,
        # warmup_steps = 100,
        warmup_ratio = 0.06,
        num_train_epochs = 2,
        # max_steps = 60,
        learning_rate = 1e-5,
        # learning_rate = 1e-5,
        # learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_hf",
        # optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",  # "linear",
        seed = 777,
        output_dir = "outputs_sft",
        save_strategy = "no",
        # save_steps = 1000,
        # save_total_limit = 2,
        # report_to="none",
        report_to = "wandb",
        run_name = run_name,
    ),
)

Map (num_proc=16):   0%|          | 0/4511 [00:00<?, ? examples/s]

In [30]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H100 80GB HBM3. Max memory = 79.109 GB.
14.689 GB of memory reserved.


In [ ]:
try:
    trainer_stats = trainer.train()
except Exception as e:
    print(e)
    

In [ ]:
import wandb
# wandb 로깅 종료
wandb.finish()

## 5. 모델 저장

In [34]:
# # LoRA Adapter 저장
# model.save_pretrained("lora_model")
# tokenizer.save_pretrained("lora_model")

In [35]:
# # # Merged model 저장
# local_model_save_path = "sft-v7.2-1206-hq2-prc3"
# model.save_pretrained_merged(local_model_save_path, tokenizer, save_method = "merged_16bit",)

In [ ]:
# Merged model 업로드
import requests

deploy_name = 'lsw0570168/krx-q25-7b-sft-v7.2-1206-hq2-prc6'
model.push_to_hub_merged(deploy_name, tokenizer, save_method = "merged_16bit", token = hf_token) # 개인 huggingface token을 사용하여 업로드할 수 있습니다.

Unsloth: You are pushing to hub, but you passed your HF username = lsw0570168.
We shall truncate lsw0570168/krx-q25-7b-sft-v7.2-1206-hq2-prc6 to krx-q25-7b-sft-v7.2-1206-hq2-prc6


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 671.96 out of 1007.38 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 993.52it/s]


Unsloth: Saving to organization with address lsw0570168/krx-q25-7b-sft-v7.2-1206-hq2-prc6
Unsloth: Saving tokenizer... Done.
Unsloth: Saving to organization with address lsw0570168/krx-q25-7b-sft-v7.2-1206-hq2-prc6
Unsloth: Uploading all files... Please wait...


  0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

## 6. 모델 추론 및 업로드

In [37]:
FastLanguageModel.for_inference(model)

qq = '''\
다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.

### 질문:
의무 위반의 효과로 보험자가 그 보험계약을 해지할 수 있다고 상법에 명시되지 않은 것을 고르시오. (다툼이 있는 경우 판례에 따름)**

### 선택지:
A. 보험계약 당시, 보험계약자나 피보험자가 고의 또는 중대한 과실로 인해 중요한 사항을 고지하지 않거나 부실하게 고지한 경우  
B. 보험기간 중, 보험계약자 또는 피보험자가 사고 발생 위험이 현저히 변경되거나 증가된 사실을 알았음에도 지체 없이 보험자에게 통지하지 않은 경우  
C. 보험계약자, 피보험자 또는 보험수익자가 보험사고의 발생을 알았을 때, 이를 지체 없이 보험자에게 통지하지 않은 경우  
D. 보험기간 중, 보험계약자, 피보험자 또는 보험수익자의 고의 또는 중대한 과실로 인해 사고 발생 위험이 현저히 변경되거나 증가된 경우
### 정답:
'''

inputs = tokenizer(
[
    qq,
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.

### 질문:
의무 위반의 효과로 보험자가 그 보험계약을 해지할 수 있다고 상법에 명시되지 않은 것을 고르시오. (다툼이 있는 경우 판례에 따름)**

### 선택지:
A. 보험계약 당시, 보험계약자나 피보험자가 고의 또는 중대한 과실로 인해 중요한 사항을 고지하지 않거나 부실하게 고지한 경우  
B. 보험기간 중, 보험계약자 또는 피보험자가 사고 발생 위험이 현저히 변경되거나 증가된 사실을 알았음에도 지체 없이 보험자에게 통지하지 않은 경우  
C. 보험계약자, 피보험자 또는 보험수익자가 보험사고의 발생을 알았을 때, 이를 지체 없이 보험자에게 통지하지 않은 경우  
D. 보험기간 중, 보험계약자, 피보험자 또는 보험수익자의 고의 또는 중대한 과실로 인해 사고 발생 위험이 현저히 변경되거나 증가된 경우
### 정답:
#### Step 1: 문제의 의도 파악 및 유형 분류
- **문제의 의도**: 보험계약에서의 의무 위반으로 인해 보험자가 보험계약을 해지할 수 있는 경우를 식별하는 문제입니다.
- **핵심 개념**: 보험계약의 해지 사유, 의무 위반, 고지의무, 통지의무
- **문제 유형**: 분류 문제 - 주어진 보기 중 상법에 명시되지 않은 보험계약 해지 사유를 찾는 문제입니다.

#### Step 2: 풀이 계획 수립
- 각 보기에서 제시된 상황이 상법에 명시된 보험계약 해지 사유인지 확인합니다.
- 상법에 명시되지 않은 경우를 찾기 위해 각 보기를 검토합니다.

#### Step 3: 각 보기별 문제 풀이 및 검토
- **A. 보험계약 당시, 보험계약자나 피보험자가 고의 또는 중대한 과실로 인해 중요한 사항을 고지하지 않거나 부실하게 고지한 경우**
  - 상법 제643조에 따르면, 보험계약자나 피보험자가 중요한 사항을 고지하지 않거나 부실하게 고지한 경우, 보험자는 계약을 해지할 수 있습니다. 따라서 이 경우는 상법에 명시되어 있습니다.

- **B. 보험기간 중, 보험계약자 또는 피보험자가 사

In [39]:
# qq = "일본의 2023년 6월 개정 자금결제법의 주요 변경 사항을 설명하고, 이러한 변경이 일본의 금융시장 및 국제 금융 규제에 미치는 영향을 논의하시오."
# qq = "숫자를 10으로 나눈 값은 6입니다. 윤기는 특정 숫자로부터 15를 빼서 결과를 얻었습니다. 그가 얻은 결과는 무엇일까요?"
# qq = "정국, 지민, 석진, 태형, 남준이 나란히 서 있습니다. 정국은 지민의 오른쪽에, 석진은 지민의 왼쪽에 서 있습니다. 또한 남준은 석진 왼쪽에, 정국은 태형 왼쪽에 서 있습니다. 가장 오른쪽에 서 있는 사람은 누구인가요?"
# qq = "숫자 2, 3, 5, 7을 무작위로 배열하여 4자리 숫자를 만듭니다. 이 숫자가 홀수일 확률은 얼마입니까? 답을 공통 분수로 표현하세요."
# qq = "표준 6면 주사위 두 개를 굴립니다. 주사위를 굴려 나온 합이 완벽한 제곱수일 확률은 얼마입니까?"
# qq = """\
# 'question': '원가, 품질, 서비스, 속도와 같은 주요 성과측정치의 극적인 개선을 위해 업무프로세스를 급진적으로 재설계하는 것으로 정의할 수 있는 것은 무엇인가?',
#  'A': 'BSC (Balanced Scorecard)',
#  'B': 'BPR (business process reengineering)',
#  'C': 'CALS (Commerce At Light Speed)',
#  'D': 'EIS (Executive Information System)',
# """
# qq="""\
# 세 변의 길이가 각각 10cm, 8cm, 6cm인 직각 삼각형이 있습니다. 둘러싸인 원의 반지름의 길이는 얼마입니까?
# """
# qq = "아메리칸 콜 옵션의 가격 책정에 대해 논의하겠습니다. 다음 질문에 답하십시오. 비배당 주식에 대한 아메리칸 콜 옵션은 유럽식 콜 옵션과 비교할 때 조기 실행이 최적이 아니라고 주장하는 여러 자료가 있습니다. 그러나 해당 주식이 배당금을 지급하지 않을 경우 유럽식 옵션과 아메리칸 옵션이 동일하다는 주장도 존재합니다. 이와 관련하여 아메리칸 콜 옵션의 가격을 다음의 두 가지 표현으로 나타낼 수 있다고 가정하십시오: 1. 유럽식 콜 옵션 가격의 경우: $$V_n(\omega) = \frac{1}{1+r} (PV_{n+1}(\omega H) + QV_{n+1}(\omega T))$$ 2. 아메리칸 콜 옵션 가격의 경우: $$V_n(\omega) = max(S(\omega) - K, \frac{1}{1+r} (PV_{n+1}(\omega H) + QV_{n+1}(\omega T)))$$ 이때, 비배당 주식에 대해 깊은 인더머니 상태의 아메리칸 콜 옵션이 만기 전에 존재한다고 가정할 때, 이 옵션의 가격은 얼마인지 설명하십시오. 또한 아메리칸 콜 옵션의 조기 실행이 최적이 아닐 경우, 왜 (2)의 식을 사용하여 이 옵션에 대한 가격을 정해야 하는지 그 이유를 논의하세요."
qq = "다음 조건을 만족하는 아시안 옵션 가격의 경계 조건에 대해 설명하시오. 주어진 확률적 과정은 다음과 같습니다: \[ dX_t=(-a_1X_t+\gamma_t)dt+dW_t \] 또한, 다음과 같은 정의가 주어집니다: \[ Y_t=\int_{0}^{t}X_u du \] 주어진 경계 조건은 다음과 같습니다: \[ v(t,0,y)=e^{-r(T-t)}\max\left(\frac{y}{T}-K,0\right) \] 이 상황에서 \(X_t=0\)이고 \(Y_t=y\)일 때, \(X_u=0\)가 성립하며 \(u \in [t,T]\)의 모든 범위에서 \(Y_u\)는 상수로 유지됩니다. 이로 인해 \(Y_T=y\)가 성립하고, 따라서 시간 \(t\)에서 아시안 콜 옵션의 가치는 \(e^{-r(T-t)}\max\left(\frac{y}{T}-K,0\right)\)로 표현된다는 주장을 보이시오. 이러한 주장을 통해 주어진 경계 조건의 의미를 논증하시오."

inputs = tokenizer(
[
    qq,
], return_tensors = "pt").to("cuda")

print('------<v1>------')

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

# print('------<v11>------')
      
# outputs = model2.generate(**inputs, max_new_tokens = 1024, use_cache = True)
# print(tokenizer.batch_decode(outputs)[0])

------<v1>------
ight)\)로 표현된다는 주장을 보이시오. 이러한 주장을 통해 주어진 경계 조건의 의미를 논증하시오. \(u \in [t,T]\)의 모든 범위에서 \(Y_u\)는 상수로 유지됩니다. 이로 인해 \(Y_T=y\)가 성립하고, 따라서 시간 \(t\)에서 아시안 콜 옵션의 가치는 \(e^{-r(T-t)}\max\left(
rac{y}{T}-K,0)}\max\left(
rac{y}{T}-K,0

---

**주어진 조건과 아시안 옵션 가격의 경계 조건에 대한 논의**

---

**1. 문제의 의도와 주어진 조건의 이해**

- **확률적 과정**: 주어진 확률적 과정은 다음과 같습니다:
  \[
  dX_t = (-a_1 X_t + \gamma_t) dt + dW_t
  \]
  여기서 \(X_t\)는 주가나 자산 가격을 나타내며, \(a_1\)은 자산의 감쇠율, \(\gamma_t\)는 시간에 따른 자산의 수익률, \(dW_t\)는 Wiener 과정(백색 잡음)입니다.

- **정의**: 
  \[
  Y_t = \int_{0}^{t} X_u du
  \]
  이는 시간 \(t\)까지의 자산 가격의 누적값을 나타냅니다.

- **경계 조건**:
  \[
  v(t,0,y) = e^{-r(T-t)} \max\left( \frac{y}{T} - K, 0 \right)
  \]
  여기서 \(v(t,0,y)\)는 시간 \(t\)에서 자산 가격이 \(X_t = 0\)이고 누적값이 \(Y_t = y\)일 때의 옵션 가격을 나타냅니다. \(K\)는 행사가격, \(r\)은 무위험 이자율, \(T\)는 만기입니다.

- **조건**: 
  - \(X_t = 0\)이고 \(Y_t = y\)일 때, \(X_u = 0\)이 성립하며, \(u \in [t, T]\)의 모든 범위에서 \(Y_u\)는 상수로 유지됩니다.
  - 따라서 \(Y_T = y\)가 성립합니다.

**2. 아시안 콜 옵션의 가치 계산**

- **아시안 콜 옵션의 정의**: 아시

In [ ]:
# del model
# torch.cuda.empty_cache()